# Import Packages

In [25]:
import tabula
from re import sub

# Import Data

In [36]:
pdf_path="data/discover_10K_2021.pdf"
pdf_data=tabula.read_pdf(pdf_path, pages=61)[0]

In [7]:
pdf_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,NaN,NaN,Average,NaN,NaN,NaN,NaN,Average,NaN,NaN,NaN,NaN,Average,NaN,NaN,NaN,NaN
1,NaN,NaN,Balance,Yield/Rate,NaN,Interest,NaN,Balance,Yield/Rate,NaN,Interest,NaN,Balance,Yield/Rate,NaN,NaN,Interest
2,Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Interest-earning assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cash and cash equivalents,$,"14,236",0.13,%,$ 18,$,"11,348",0.30,%,$ 35,$,"9,667",2.27,%,$,219
5,Restricted cash,NaN,695,0.03,%,NM,NaN,438,0.45,%,2,NaN,620,2.24,%,NaN,14
6,Other short-term investments,NaN,176,0.12,%,NM,NaN,"2,677",0.14,%,4,NaN,754,2.66,%,NaN,20
7,Investment securities,NaN,"8,713",2.09,%,182,NaN,"11,431",2.21,%,252,NaN,"7,603",2.35,%,NaN,179
8,Loan receivables(1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Credit card loans(2),NaN,"69,365",12.57,%,"8,717",NaN,"71,447",12.58,%,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"


# Data Engineering

## Extract Column Name from the first two columns

In [29]:
def replace_special_char(input_string:str,char_replacement=" "):
    return sub('[^a-zA-Z0-9 \n\.]', char_replacement, input_string)
def join_string_from_list(input_list:list,char_join=" "):
    return char_join.join(input_list)
def snake_case(input_string:str)->str:
  return '_'.join(
    sub('([A-Z][a-z]+)', r' \1',
    sub('([A-Z]+)', r' \1',
    input_string.replace('-', ' '))).split()).lower()

In [32]:
row_desc_ser=pdf_data.iloc[[0,1]].agg(lambda x: list(x.dropna())).apply(lambda x: snake_case(replace_special_char(join_string_from_list(x))))

row_desc_ser

Unnamed: 0                    
Unnamed: 1                    
Unnamed: 2     average_balance
2021                yield_rate
Unnamed: 3                    
Unnamed: 4            interest
Unnamed: 5                    
Unnamed: 6     average_balance
2020                yield_rate
Unnamed: 7                    
Unnamed: 8            interest
Unnamed: 9                    
Unnamed: 10    average_balance
2019                yield_rate
Unnamed: 11                   
Unnamed: 12                   
Unnamed: 13           interest
dtype: object

In [37]:
# Delete the first two rows
pdf_data=pdf_data.drop([0,1]).reset_index(drop=True)
pdf_data

,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2020,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,2019,Unnamed: 11,Unnamed: 12,Unnamed: 13
0,Assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Interest-earning assets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Cash and cash equivalents,$,"14,236",0.13,%,$ 18,$,"11,348",0.30,%,$ 35,$,"9,667",2.27,%,$,219
3,Restricted cash,NaN,695,0.03,%,NM,NaN,438,0.45,%,2,NaN,620,2.24,%,NaN,14
4,Other short-term investments,NaN,176,0.12,%,NM,NaN,"2,677",0.14,%,4,NaN,754,2.66,%,NaN,20
5,Investment securities,NaN,"8,713",2.09,%,182,NaN,"11,431",2.21,%,252,NaN,"7,603",2.35,%,NaN,179
6,Loan receivables(1),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Credit card loans(2),NaN,"69,365",12.57,%,"8,717",NaN,"71,447",12.58,%,"8,985",NaN,"72,740",13.32,%,NaN,"9,690"
8,Private student loans,NaN,"10,057",7.38,%,742,NaN,"9,890",7.63,%,754,NaN,"9,559",8.54,%,NaN,817
9,Personal loans,NaN,"6,945",12.64,%,878,NaN,"7,406",12.93,%,958,NaN,"7,522",13.07,%,NaN,983


In [40]:
pdf_data["Unnamed: 4"].apply(lambda x: )

0         NaN
1         NaN
2        $ 18
3          NM
4          NM
5         182
6         NaN
7       8,717
8         742
9         878
10        114
11     10,451
12     10,651
13        NaN
14        NaN
15        NaN
16        NaN
17        NaN
18        NaN
19        440
20         43
21        178
22        661
23        NaN
24         NM
25        102
26        371
27        473
28      1,134
29        NaN
30        NaN
31    $ 9,517
32        NaN
33        NaN
34        NaN
Name: Unnamed: 4, dtype: object